In [2]:
import pandas as pd
import numpy as np
import xlrd
from pandas import DataFrame
import datetime
import matplotlib.pyplot as plt
from matplotlib import style
import os



files = os.listdir("house-office-expenditures-with-readme")
#locations = ["house-office-expenditures-with-readme/{}".format(i) for i in files]
df = pd.concat([pd.read_csv(i, encoding = "latin1",  error_bad_lines=False) for i in files])
df= df[["BIOGUIDE_ID", "OFFICE", "CATEGORY", "PAYEE", "START DATE", "END DATE", "AMOUNT", "PURPOSE","YEAR"]]
df.to_csv("csvfile.csv", sep=",")



#file_location = '2015Q2-house-disburse-detail-updated.csv'
#df = pd.read_csv(file_location)

#df.tail()

/Users/arminkhayyer/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/arminkhayyer/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
b'Skipping line 5: expected 1 fields, saw 2\n'


In [3]:
def convert_to_float(row):
    string= str(row["AMOUNT"]).replace(",", "")
    return float(string)
    

df["AMOUNT"] = df.apply(lambda row: convert_to_float(row), axis =1)




In [4]:
df.AMOUNT.sum()

13660703793.310001

In [5]:
df = df.loc[df["START DATE"] != '   ']
df['START DATE'].replace('', np.nan, inplace=True)
print(df['START DATE'].isnull().sum())
#df = df.loc[df["START DATE"] != "", :]
#df = df.loc[df["END DATE"] != "", :]
df.dropna(subset=['START DATE'], inplace=True)



df["START DATE"] = pd.to_datetime(df["START DATE"])
start_date = pd.to_datetime("1/1/2010")
end_date =  pd.to_datetime("12/31/2016")
mean = df.loc[(df["START DATE"] >= start_date) & (df["START DATE"] <= end_date)&(df["AMOUNT"]>0), "AMOUNT"].mean()
print(mean)

34
3146.7727500697324


In [6]:
# the data set is for 2015 second quarter, so there is no data point for 2016 in this dataset 
new_df = df.loc[(df["CATEGORY"] =="PERSONNEL COMPENSATION")&(df["START DATE"].dt.year ==2016), :]
new_df.groupby(["PAYEE"]).mean()["AMOUNT"].max()

50970.33

In [7]:
#coverage period


df["END DATE"] = pd.to_datetime(df["END DATE"], errors="coerce")
df.dropna(subset=['END DATE'], inplace=True)
df["coverage period"] = (df["END DATE"] - df["START DATE"]).dt.days

print(np.std(df.loc[df["AMOUNT"] >0 , "coverage period"]))



61.443788552238495


In [19]:

groups = df.loc[df["START DATE"].dt.year==2016, :].groupby(["OFFICE"])["AMOUNT"].max()
purpose = df.loc[df.AMOUNT ==groups.max(), "PURPOSE"]


total = df.loc[df["START DATE"].dt.year==2016, "AMOUNT"].sum()

purpose_total = df.loc[(df["PURPOSE"] == "FERS")&(df["START DATE"].dt.year ==2016), "AMOUNT"].sum()

fraction = purpose_total/total

print(fraction)
       


0.06917102803962792
